# Visualización Interactiva SIMCE
Análisis de puntajes SIMCE por región y comuna con drill-down interactivo

In [55]:
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display, HTML
import numpy as np
import ipywidgets as widgets
from pathlib import Path
import unicodedata
import re

## Cargar Datasets

In [56]:
# Cargar datos desde carpetas con múltiples años

def cargar_curso(dir_path, curso, col_lectura, col_matematica):
    archivos = sorted(Path(dir_path).glob('*.csv'))
    if not archivos:
        raise FileNotFoundError(f"No se encontraron CSV en {dir_path}")
    dataframes = []
    for archivo in archivos:
        df_tmp = pd.read_csv(archivo)
        df_tmp['curso'] = curso
        dataframes.append(df_tmp)
    df_curso = pd.concat(dataframes, ignore_index=True)
    cols = ['agno', 'nom_reg_rbd', 'nom_com_rbd', col_lectura, col_matematica, 'curso']
    df_filtrado = df_curso[cols].copy()
    df_filtrado['promedio'] = df_filtrado[[col_lectura, col_matematica]].mean(axis=1)
    df_filtrado = df_filtrado.rename(columns={
        col_lectura: 'lectura',
        col_matematica: 'matematica'
    })
    return df_filtrado

# Directorios con CSV por curso
dir_2m = Path('datasets/2do medio')
dir_4b = Path('datasets/4tobasico')

df_2m_filtered = cargar_curso(dir_2m, '2° Medio', 'prom_lect2m_rbd', 'prom_mate2m_rbd')
df_4b_filtered = cargar_curso(dir_4b, '4° Básico', 'prom_lect4b_rbd', 'prom_mate4b_rbd')

df = pd.concat([df_2m_filtered, df_4b_filtered], ignore_index=True)
df = df.dropna(subset=['promedio', 'nom_reg_rbd', 'nom_com_rbd'])

available_years = sorted(df['agno'].unique().tolist())
print(f"Total de registros: {len(df)}")
print(f"Años disponibles: {available_years}")
print(f"Regiones: {df['nom_reg_rbd'].nunique()}")

Total de registros: 78406
Años disponibles: [2014, 2015, 2016, 2017, 2018, 2022, 2023, 2024]
Regiones: 79


In [57]:
# Normalizar nombres de regiones con notación común

def _strip_accents(texto: str) -> str:
    return ''.join(
        c for c in unicodedata.normalize('NFKD', texto)
        if not unicodedata.combining(c)
    )

STOPWORDS = {"REGION", "DE", "DEL", "LA", "LAS", "LOS", "Y", "E", "EL", "GENERAL", "CARLOS", "DON", "DO", "DA"}
TOKEN_SYNONYMS = {
    "AISEN": "AYSEN",
    "HIGGINS": "LIBERTADOR",
    "OHIGGINS": "LIBERTADOR",
    "SANTIAGO": "METROPOLITANA",
    "RM": "METROPOLITANA"
}

REGION_RULES = [
    ({"ARICA", "PARINACOTA"}, "Región de Arica y Parinacota (XV)"),
    ("TARAPACA", "Región de Tarapacá (I)"),
    ("ANTOFAGASTA", "Región de Antofagasta (II)"),
    ("ATACAMA", "Región de Atacama (III)"),
    ("COQUIMBO", "Región de Coquimbo (IV)"),
    ("VALPARAISO", "Región de Valparaíso (V)"),
    ("METROPOLITANA", "Región Metropolitana de Santiago (RM)"),
    ("LIBERTADOR", "Región del Libertador General Bernardo O’Higgins (VI)"),
    ("MAULE", "Región del Maule (VII)"),
    ("NUBLE", "Región de Ñuble (XVI)"),
    ("BIOBIO", "Región del Biobío (VIII)"),
    ("ARAUCANIA", "Región de La Araucanía (IX)"),
    ("RIOS", "Región de Los Ríos (XIV)"),
    ("LAGOS", "Región de Los Lagos (X)"),
    ("AYSEN", "Región de Aysén del General Carlos Ibáñez del Campo (XI)"),
    ("MAGALLANES", "Región de Magallanes y de la Antártica Chilena (XII)")
]

# Aplicar normalización

def _tokenize_region(nombre: str) -> set:
    cleaned = re.sub(r"[^A-Z ]", " ", _strip_accents(nombre).upper())
    tokens = set()
    for token in cleaned.split():
        if token in STOPWORDS:
            continue
        token = TOKEN_SYNONYMS.get(token, token)
        tokens.add(token)
    return tokens

unmatched_regiones = set()

def normalizar_region(nombre):
    if pd.isna(nombre):
        return nombre
    original = str(nombre).strip()
    tokens = _tokenize_region(original)
    for key, canonical in REGION_RULES:
        required = key if isinstance(key, set) else {key}
        if required.issubset(tokens):
            return canonical
    unmatched_regiones.add(original)
    return original.title()

regiones_antes = df['nom_reg_rbd'].nunique()
df['nom_reg_rbd'] = df['nom_reg_rbd'].apply(normalizar_region)
print(f"Regiones únicas antes: {regiones_antes} -> después: {df['nom_reg_rbd'].nunique()}")
if unmatched_regiones:
    print("Regiones sin normalizar (revisar diccionario):")
    for reg in sorted(unmatched_regiones):
        print(f" - {reg}")

Regiones únicas antes: 79 -> después: 16


## Calcular Promedios Regionales

In [58]:
# Agrupar por región, año y curso
df_regional = df.groupby(['agno', 'nom_reg_rbd', 'curso']).agg({
    'promedio': 'mean',
    'lectura': 'mean',
    'matematica': 'mean'
}).reset_index()

# Agrupar por comuna, año y curso
df_comunal = df.groupby(['agno', 'nom_reg_rbd', 'nom_com_rbd', 'curso']).agg({
    'promedio': 'mean',
    'lectura': 'mean',
    'matematica': 'mean'
}).reset_index()

print("Datos regionales preparados")
df_regional.head()

Datos regionales preparados


,agno,nom_reg_rbd,curso,promedio,lectura,matematica
0,2014,Región Metropolitana de Santiago (RM),2° Medio,260.034962,252.372605,267.697318
1,2014,Región Metropolitana de Santiago (RM),4° Básico,254.812919,258.706935,250.949077
2,2014,Región de Antofagasta (II),2° Medio,266.294521,260.178082,272.410959
3,2014,Región de Antofagasta (II),4° Básico,254.980769,258.884615,251.372093
4,2014,Región de Arica y Parinacota (XV),2° Medio,250.151515,245.181818,255.121212


## Visualización Interactiva con Drill-Down

In [ ]:
class VisualizacionSimce:
    def __init__(self, df_regional, df_comunal):
        self.df_regional = df_regional
        self.df_comunal = df_comunal
        self.current_region = None
        self.current_curso = '2° Medio'
        self.metric_labels = {
            'promedio': 'Promedio General',
            'matematica': 'Promedio Matemática',
            'lectura': 'Promedio Lenguaje'
        }
        self.current_metric = 'promedio'
        self.fig = go.FigureWidget()
        self.btn_volver = widgets.Button(
            description='← Volver',
            icon='arrow-left',
            button_style='info',
            layout=widgets.Layout(width='140px', display='none')
        )
        self.btn_volver.on_click(self._handle_back_button)
        self.selector_curso = widgets.ToggleButtons(
            options=['2° Medio', '4° Básico'],
            value='2° Medio',
            description='Curso:',
            button_style='primary',
            layout=widgets.Layout(width='320px')
        )
        self.selector_curso.observe(self._handle_curso_change, names='value')
        self.selector_metric = widgets.ToggleButtons(
            options=[
                ('Promedio General', 'promedio'),
                ('Promedio Matemática', 'matematica'),
                ('Promedio Lenguaje', 'lectura')
            ],
            value='promedio',
            description='Indicador:',
            button_style='',
            layout=widgets.Layout(width='480px')
        )
        self.selector_metric.observe(self._handle_metric_change, names='value')
        self.title_widget = widgets.HTML(
            "<h3 style='margin:0 0 8px 0;'>Visualización SIMCE Interactiva</h3>"
        )
        self.controls = widgets.VBox([
            self.title_widget,
            widgets.HBox([
                self.selector_curso,
                widgets.Box([self.btn_volver], layout=widgets.Layout(padding='0 0 0 16px'))
            ]),
            self.selector_metric
        ])
        self.container = widgets.VBox([self.controls, self.fig])
        self.year_ticks = sorted(self.df_regional['agno'].unique().tolist())
        self.year_labels = [str(y) for y in self.year_ticks]
        self.x_padding = 7.5  # 1.5x de 5 (5 * 1.5 = 7.5) para más separación entre puntajes
        self.x_zoom_padding = 3  # Padding reducido para zoom inicial más cerrado
        self.metric_ranges = {
            col: [
                float(self.df_regional[col].min()) - self.x_zoom_padding,
                float(self.df_regional[col].max()) + self.x_zoom_padding
            ]
            for col in ['promedio', 'matematica', 'lectura']
        }
        self.y_range = [self.year_ticks[0] - 0.5, self.year_ticks[-1] + 0.5]
    
    def crear_grafico_regional(self, curso='2° Medio'):
        """Mostrar vista regional y resetear navegación"""
        self.current_curso = curso
        if self.selector_curso.value != curso:
            self.selector_curso.value = curso
        self.current_region = None
        df_curso = self.df_regional[self.df_regional['curso'] == curso]
        metric_col = self.current_metric
        metric_label = self.metric_labels[metric_col]
        self.fig = go.FigureWidget()
        
        for region in df_curso['nom_reg_rbd'].unique():
            df_reg = df_curso[df_curso['nom_reg_rbd'] == region]
            self.fig.add_trace(go.Scatter(
                x=df_reg[metric_col],
                y=df_reg['agno'],
                mode='markers',
                name=region,
                marker=dict(size=12, opacity=0.7),
                hovertemplate='<b>%{fullData.name}</b><br>' +
                              'Año: %{y}<br>' +
                              f'{metric_label}: %{{x:.1f}}<br>' +
                              '<extra></extra>'
            ))
        
        self.fig.update_layout(
            title=dict(
                text=f'{metric_label} SIMCE {curso} por Región (Click para ver comunas)',
                y=0.97,
                yanchor='top',
                pad=dict(b=15)
            ),
            xaxis=dict(
                title=metric_label,
                range=self.metric_ranges[metric_col],
                autorange=False
            ),
            yaxis=dict(
                title='Año',
                tickmode='array',
                tickvals=self.year_ticks,
                ticktext=self.year_labels,
                range=self.y_range,
                autorange=False
            ),
            hovermode='closest',
            height=650,
            width=1200,
            margin=dict(l=80, r=40, b=120, t=160),
            showlegend=True,
            legend=dict(
                orientation='h',
                yanchor='top',
                y=-0.15,
                xanchor='center',
                x=0.5,
                itemclick='toggleothers',
                itemdoubleclick='toggle'
            ),
            dragmode='pan'
        )
        
        self._attach_trace_handlers(self._on_region_click)
        self._set_button_visibility(False)
        self._refresh_container()
        return self.container
    
    def _on_region_click(self, trace, points, selector):
        if points.point_inds:
            region = trace.name
            self.crear_grafico_comunal(region, self.current_curso)
    
    def crear_grafico_comunal(self, region, curso):
        """Mostrar vista comunal con un único trace WebGL para ganar rendimiento"""
        self.current_region = region
        df_curso = self.df_comunal[
            (self.df_comunal['nom_reg_rbd'] == region) &
            (self.df_comunal['curso'] == curso)
        ]
        metric_col = self.current_metric
        metric_label = self.metric_labels[metric_col]
        self.fig = go.FigureWidget()
        
        if df_curso.empty:
            self.fig.add_annotation(text='Sin datos para esta región/curso', showarrow=False)
            tickvals = self.year_ticks
            ticktext = self.year_labels
            x_range = self.metric_ranges[metric_col]
        else:
            self.fig.add_trace(go.Scattergl(
                x=df_curso[metric_col],
                y=df_curso['agno'],
                mode='markers',
                marker=dict(
                    size=9,
                    opacity=0.65,
                    color=df_curso[metric_col],
                    colorscale='Viridis',
                    showscale=True,
                    colorbar=dict(title=metric_label)
                ),
                text=df_curso['nom_com_rbd'],
                hovertemplate='<b>%{text}</b><br>' +
                              'Año: %{y}<br>' +
                              f'{metric_label}: %{{x:.1f}}<br>' +
                              '<extra></extra>'
            ))
            region_years = sorted(df_curso['agno'].unique().tolist())
            tickvals = region_years if region_years else self.year_ticks
            ticktext = [str(y) for y in tickvals]
            x_range = [
                float(df_curso[metric_col].min()) - self.x_padding,
                float(df_curso[metric_col].max()) + self.x_padding
            ]
        
        self.fig.update_layout(
            title=dict(
                text=f'Comunas de {region} - {curso} ({metric_label})',
                y=0.97,
                yanchor='top',
                pad=dict(b=10)
            ),
            xaxis=dict(
                title=metric_label,
                range=x_range,
                autorange=False
            ),
            yaxis=dict(
                title='Año',
                tickmode='array',
                tickvals=tickvals,
                ticktext=ticktext,
                range=[tickvals[0] - 0.5, tickvals[-1] + 0.5] if tickvals else self.y_range,
                autorange=False
            ),
            height=650,
            width=1200,
            margin=dict(l=80, r=40, b=100, t=150),
            showlegend=False,
            dragmode='pan'
        )
        
        self._attach_trace_handlers(self._on_back_click)
        self._set_button_visibility(True)
        self._refresh_container()
        return self.container
    
    def _on_back_click(self, trace, points, selector):
        self.crear_grafico_regional(self.current_curso)
    
    def _handle_back_button(self, _):
        self.crear_grafico_regional(self.current_curso)
    
    def _handle_curso_change(self, change):
        nuevo_curso = change['new']
        if nuevo_curso != self.current_curso:
            self.crear_grafico_regional(nuevo_curso)
    
    def _handle_metric_change(self, change):
        nuevo_metric = change['new']
        if nuevo_metric != self.current_metric:
            self.current_metric = nuevo_metric
            if self.current_region:
                self.crear_grafico_comunal(self.current_region, self.current_curso)
            else:
                self.crear_grafico_regional(self.current_curso)
    
    def _attach_trace_handlers(self, handler):
        for trace in self.fig.data:
            trace.on_click(handler)
    
    def _set_button_visibility(self, visible):
        self.btn_volver.layout.display = 'inline-flex' if visible else 'none'
        self.btn_volver.disabled = not visible
    
    def _refresh_container(self):
        self.container.children = (self.controls, self.fig)
    
# Crear instancia
viz = VisualizacionSimce(df_regional, df_comunal)
print("Visualización creada. Ejecuta la siguiente celda para mostrar el gráfico.")

## Mostrar Gráfico Interactivo

**Instrucciones:**
- Click en cualquier punto para ver las comunas de esa región
- Click en "← Volver" o en cualquier punto de comunas para regresar a regiones
- Puedes cambiar el curso modificando el parámetro en `crear_grafico_regional()`

In [60]:
# Mostrar gráfico para 2° Medio
viz.crear_grafico_regional('2° Medio')

## Opción Alternativa: Ver 4° Básico

In [61]:
# Descomentar para ver 4° Básico
# viz.crear_grafico_regional('4° Básico')